In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from keras.layers import LSTM, Dense,Input,Layer
from keras import backend as K,Model
from numpy.random import seed
from imblearn.under_sampling import RandomUnderSampler 
seed(0)
import tensorflow as tf
tf.random.set_seed(0)

In [29]:
data = pd.read_csv('creditcard.csv', na_filter=True)

data.drop(['Time'], axis=1, inplace=True)
selected_vars =['V4','V14','V10','V16','V9','V12','V11','V17','V7','V21','V18','V8','V1','V3','V6','V2','V27','V5','V26','Class']


In [30]:
tr_data = data[selected_vars]
tr_data.shape

(284807, 20)

In [31]:
X = tr_data.drop(['Class'], axis = 'columns')
Label_Data = tr_data['Class']

In [32]:
y=tr_data['Class']

In [33]:
from sklearn.preprocessing import StandardScaler
## Standardizing the data
X=pd.DataFrame(StandardScaler().fit_transform(X))

In [34]:
X_train0,X_test,y_train0,y_test = train_test_split(X, y, test_size=0.3, shuffle=False,random_state=0)
X_train0,X_train2,y_train0,y_train2 = train_test_split(X_train0, y_train0, test_size=0.333, shuffle=False,random_state=0)
X_train0,X_train1,y_train0,y_train1 = train_test_split(X_train0, y_train0, test_size=0.5, shuffle=False,random_state=0)
from sklearn.linear_model import SGDOneClassSVM
df1_normal=X_train0[y_train0==0]
clf = SGDOneClassSVM(nu=0.02,random_state=0)
res = clf.fit_predict(df1_normal)
res[res==1] = 0
res[res==-1] = 1
ros = RandomUnderSampler(sampling_strategy=0.95)
ros.fit_resample(df1_normal,res)
index=ros.sample_indices_
X_train0.drop(X_train0.index[index], inplace=True)
y_train0.drop(y_train0.index[index], inplace=True)

df1_normal=X_train1[y_train1==1]
clf = SGDOneClassSVM(nu=0.03,random_state=0)
res = clf.fit_predict(df1_normal)
res[res==1] = 0
res[res==-1] = 1
ros = RandomUnderSampler(sampling_strategy=0.95)
ros.fit_resample(df1_normal,res)
index=ros.sample_indices_
X_train1.drop(X_train1.index[index], inplace=True)
y_train1.drop(y_train1.index[index], inplace=True)

df1_normal=X_train2[y_train2==1]
clf = SGDOneClassSVM(nu=0.03,random_state=0)
res = clf.fit_predict(df1_normal)
res[res==1] = 0
res[res==-1] = 1
ros = RandomUnderSampler(sampling_strategy=0.95)
ros.fit_resample(df1_normal,res)
index=ros.sample_indices_
X_train2.drop(X_train2.index[index], inplace=True)
y_train2.drop(y_train2.index[index], inplace=True)


In [35]:
X_train0.shape,X_train1.shape

((65508, 19), (66484, 19))

In [36]:
y_train0.shape,y_train1.shape

((65508,), (66484,))

In [37]:
X_test.shape

(85443, 19)

# N=1

In [38]:
# X_train et X_test sont des dataframe qui contient les features
train_LSTM_X0=X_train0.to_numpy()
test_LSTM_X=X_test.to_numpy()
train_LSTM_y0=y_train0.to_numpy()
test_LSTM_y=y_test.to_numpy()
w=3
X_train0=np.zeros([train_LSTM_X0.shape[0]-w+1,w, train_LSTM_X0.shape[1]])
for i in range(train_LSTM_X0.shape[0]-w+1):
    for j in range(w):
        X_train0[i][j]=train_LSTM_X0[i+j]
X_test=np.zeros([test_LSTM_X.shape[0]-w+1,w, test_LSTM_X.shape[1]])
for i in range(test_LSTM_X.shape[0]-w+1):
    for j in range(w):
        X_test[i][j]=test_LSTM_X[i+j]
## Recuperation des labels

y_train0=np.delete(train_LSTM_y0,[*range(0, w-1, 1)], 0)
y_test=np.delete(test_LSTM_y,[*range(0, w-1, 1)], 0)

train_LSTM_X0=X_train0
test_LSTM_X=X_test
train_LSTM_y0=y_train0
test_LSTM_y=y_test

In [39]:
inputs=Input((w,19))
x1=LSTM(50)(inputs)
outputs=Dense(1,activation='sigmoid')(x1)
model0=Model(inputs,outputs)

In [40]:
model0.compile(loss='binary_crossentropy', optimizer='adam', metrics=['Precision','Recall'])

In [41]:
history0=model0.fit(train_LSTM_X0, train_LSTM_y0,epochs=5,batch_size=250,validation_data=(test_LSTM_X, test_LSTM_y))

Epoch 1/5
263/263 [==============================] - 8s 15ms/step - loss: 0.1181 - precision: 0.0176 - recall: 0.1687 - val_loss: 0.1178 - val_precision: 0.0209 - val_recall: 0.2963
Epoch 2/5
263/263 [==============================] - 3s 10ms/step - loss: 0.0058 - precision: 0.7319 - recall: 0.6084 - val_loss: 0.0493 - val_precision: 0.3514 - val_recall: 0.6019
Epoch 3/5
263/263 [==============================] - 2s 9ms/step - loss: 0.0045 - precision: 0.7669 - recall: 0.7530 - val_loss: 0.0261 - val_precision: 0.8667 - val_recall: 0.7222
Epoch 4/5
263/263 [==============================] - 2s 9ms/step - loss: 0.0039 - precision: 0.7841 - recall: 0.8313 - val_loss: 0.0172 - val_precision: 0.9432 - val_recall: 0.7685
Epoch 5/5
263/263 [==============================] - 2s 9ms/step - loss: 0.0036 - precision: 0.7901 - recall: 0.8614 - val_loss: 0.0117 - val_precision: 0.9432 - val_recall: 0.7685


In [42]:
# predict probabilities for test set
yhat_probs0 = model0.predict(test_LSTM_X, verbose=0)
# reduce to 1d array
yhat_probs0 = yhat_probs0[:, 0]> 0.5

In [43]:
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_LSTM_y, yhat_probs0)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_LSTM_y, yhat_probs0)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_LSTM_y, yhat_probs0)
print('Recall: %f' % recall)
f1 = f1_score(test_LSTM_y, yhat_probs0)
print('f1: %f' % f1)

Accuracy: 0.999649
Precision: 0.943182
Recall: 0.768519
f1: 0.846939


# N=2

In [44]:
# X_train et X_test sont des dataframe qui contient les features
train_LSTM_X1=X_train1.to_numpy()
train_LSTM_y1=y_train1.to_numpy()
w=3
X_train1=np.zeros([train_LSTM_X1.shape[0]-w+1,w, train_LSTM_X1.shape[1]])
for i in range(train_LSTM_X1.shape[0]-w+1):
    for j in range(w):
        X_train1[i][j]=train_LSTM_X1[i+j]

y_train1=np.delete(train_LSTM_y1,[*range(0, w-1, 1)], 0)
train_LSTM_X1=X_train1
train_LSTM_y1=y_train1

In [45]:
inputs=Input((w,19))
x1=LSTM(50)(inputs)
outputs=Dense(1,activation='sigmoid')(x1)
model1=Model(inputs,outputs)

In [46]:
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['Precision','Recall'])

In [47]:
history1=model1.fit(train_LSTM_X1, train_LSTM_y1,epochs=11,batch_size=250,validation_data=(test_LSTM_X, test_LSTM_y))

Epoch 1/11
266/266 [==============================] - 6s 14ms/step - loss: 0.1034 - precision: 0.0062 - recall: 0.0645 - val_loss: 0.1267 - val_precision: 0.0073 - val_recall: 0.1574
Epoch 2/11
266/266 [==============================] - 4s 13ms/step - loss: 0.0054 - precision: 0.9062 - recall: 0.3118 - val_loss: 0.0641 - val_precision: 0.1210 - val_recall: 0.5926
Epoch 3/11
266/266 [==============================] - 3s 13ms/step - loss: 0.0042 - precision: 0.9444 - recall: 0.5484 - val_loss: 0.0394 - val_precision: 0.3911 - val_recall: 0.6481
Epoch 4/11
266/266 [==============================] - 3s 11ms/step - loss: 0.0038 - precision: 0.9394 - recall: 0.6667 - val_loss: 0.0275 - val_precision: 0.7248 - val_recall: 0.7315
Epoch 5/11
266/266 [==============================] - 3s 10ms/step - loss: 0.0035 - precision: 0.9130 - recall: 0.6774 - val_loss: 0.0237 - val_precision: 0.7736 - val_recall: 0.7593
Epoch 6/11
266/266 [==============================] - 3s 10ms/step - loss: 0.0033 - p

In [48]:
# predict probabilities for test set
yhat_probs1 = model1.predict(test_LSTM_X, verbose=0)
# reduce to 1d array
yhat_probs1 = yhat_probs1[:, 0]> 0.5

In [49]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_LSTM_y, yhat_probs1)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_LSTM_y, yhat_probs1)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_LSTM_y, yhat_probs1)
print('Recall: %f' % recall)
f1 = f1_score(test_LSTM_y, yhat_probs1)
print('f1: %f' % f1)

Accuracy: 0.999602
Precision: 0.911111
Recall: 0.759259
f1: 0.828283


# N=3

In [50]:
# X_train et X_test sont des dataframe qui contient les features
train_LSTM_X2=X_train2.to_numpy()
train_LSTM_y2=y_train2.to_numpy()
w=3
X_train2=np.zeros([train_LSTM_X2.shape[0]-w+1,w, train_LSTM_X2.shape[1]])
for i in range(train_LSTM_X2.shape[0]-w+1):
    for j in range(w):
        X_train2[i][j]=train_LSTM_X2[i+j]

y_train2=np.delete(train_LSTM_y2,[*range(0, w-1, 1)], 0)
train_LSTM_X2=X_train2
train_LSTM_y2=y_train2

In [51]:
inputs=Input((w,19))
x1=LSTM(50)(inputs)
outputs=Dense(1,activation='sigmoid')(x1)
model2=Model(inputs,outputs)

In [52]:
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['Precision','Recall'])

In [53]:
history2=model2.fit(train_LSTM_X2, train_LSTM_y2,epochs=9,batch_size=250,validation_data=(test_LSTM_X, test_LSTM_y))

Epoch 1/9
266/266 [==============================] - 8s 19ms/step - loss: 0.1235 - precision: 0.0107 - recall: 0.1885 - val_loss: 0.0061 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/9
266/266 [==============================] - 4s 14ms/step - loss: 0.0049 - precision: 0.9688 - recall: 0.5082 - val_loss: 0.0042 - val_precision: 1.0000 - val_recall: 0.2222
Epoch 3/9
266/266 [==============================] - 3s 13ms/step - loss: 0.0037 - precision: 0.9390 - recall: 0.6311 - val_loss: 0.0035 - val_precision: 1.0000 - val_recall: 0.4444
Epoch 4/9
266/266 [==============================] - 3s 12ms/step - loss: 0.0032 - precision: 0.9000 - recall: 0.7377 - val_loss: 0.0031 - val_precision: 1.0000 - val_recall: 0.5370
Epoch 5/9
266/266 [==============================] - 3s 13ms/step - loss: 0.0029 - precision: 0.9029 - recall: 0.7623 - val_loss: 0.0030 - val_precision: 1.0000 - val_recall: 0.6204
Epoch 6/9
266/266 [==============================] - 3s 13ms/step - loss: 0.0026 -

In [54]:
# predict probabilities for test set
yhat_probs2 = model2.predict(test_LSTM_X, verbose=0)
# reduce to 1d array
yhat_probs2 = yhat_probs2[:, 0]> 0.5

In [55]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_LSTM_y, yhat_probs2)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_LSTM_y, yhat_probs2)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_LSTM_y, yhat_probs2)
print('Recall: %f' % recall)
f1 = f1_score(test_LSTM_y, yhat_probs2)
print('f1: %f' % f1)

Accuracy: 0.999625
Precision: 1.000000
Recall: 0.703704
f1: 0.826087


# FFNN

In [56]:
y_probs0 = model0.predict(train_LSTM_X0, verbose=0)
y_probs1 = model1.predict(train_LSTM_X0, verbose=0)
y_probs2 = model2.predict(train_LSTM_X0, verbose=0)
y_probs00 = model0.predict(train_LSTM_X1, verbose=0)
y_probs11 = model1.predict(train_LSTM_X1, verbose=0)
y_probs22 = model2.predict(train_LSTM_X1, verbose=0)

In [57]:
X_train0=np.concatenate((y_probs0,y_probs1,y_probs2),axis=1)
X_train1=np.concatenate((y_probs00,y_probs11,y_probs22),axis=1)
X_train=np.concatenate((X_train0,X_train1),axis=0)
y_train=np.concatenate((y_train0,y_train1),axis=0)
X_train.shape,y_train.shape

((131988, 3), (131988,))

In [58]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier().fit(X_train, y_train)

# evaluation

In [59]:
yhat_probs0 = model0.predict(test_LSTM_X, verbose=0)
yhat_probs1 = model1.predict(test_LSTM_X,verbose=0)
yhat_probs2 = model2.predict(test_LSTM_X,verbose=0)
X_test=np.concatenate((yhat_probs0,yhat_probs1,yhat_probs2),axis=1)
y_pred=clf.predict(X_test)

In [60]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_LSTM_y, y_pred)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_LSTM_y, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_LSTM_y, y_pred)
print('Recall: %f' % recall)
#F1 = 2 * (precision * recall) / (precision + recall)
f1=f1_score(test_LSTM_y, y_pred)
print('f1: %f' % f1)

Accuracy: 0.999625
Precision: 0.931818
Recall: 0.759259
f1: 0.836735
